In [ ]:
import itertools
import jax
import numpy as np
from neurallogic import hard_not, hard_and, harden, neural_logic_net

In [ ]:
isinstance(w, [True])

In [38]:
def dot(x, y, op):
    assert len(x) == len(y)
    return [op(x[i], y[i]) for i in range(len(x))]

def matmult(m, v, op):
    return [dot(row, v, op) for row in m]

In [39]:
x = [True, False]
w = [[True, True], [False, True], [True, False], [False, False]]
matmult(w, x, lambda a, b: (a, b))   

[[(True, True), (True, False)],
 [(False, True), (True, False)],
 [(True, True), (False, False)],
 [(False, True), (False, False)]]